In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
import torch
import re, json, random

# Load a pre-trained model

## Load model from HuggingFace

In [ ]:
model_name = "openlm-research/open_llama_3b_v2"

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

tokenizer.save_pretrained("openllama_f16")
model.save_pretrained("openllama_f16")

## Load model locally

In [2]:
model_name = "openllama"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    "./" + model_name + "_f16"
)
model = AutoModelForCausalLM.from_pretrained(
    "./" + model_name + "_f16",
    torch_dtype=torch.float16
).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i} ==")
    print(generated)
    print("\n")

== generation 0 ==
K. ouvrit la porte. 50% OFF End of Season Sale Up to70 off Select Styles Shop Now! And in the spirit oj a new year, weve added many more free bonuses that you can use right away just for signing up and verifying your email address with us - like our no-cost Ebook How To Choose The Best Forex Broker That Will Meet Your Needs At Every Stage Of Trading (which is yours when yo signup today). If there are any issues along this line please do not hesitate on contacting me at my e mail: brian@krakenfx.com
[index] [1189] [2096] [4336] [5407] [4563] [3206] [2715] [1784] [2520] [540]...


== generation 1 ==
K. ouvrit la porte. À l'intérieur, il ne vit que le général et un homme blanc.
"Monsieur Léon!" exclama-t-il en se redressant d'un coup. "Comment t'appelles-tu?"
Le général s'avança vers lui avec une grande pompe. Il avait les yeux bleus de la tristesse et des larmes coulèrent par-dessus son visage.
Léon s'inclina et lui tendit la main. C'était celle qui allait arriver à él

# Prepare the training data

## Load Gutenberg project

In [ ]:
gutenberg = load_dataset("manu/project_gutenberg", split="fr", streaming=False)
gutenberg.save_to_disk("gutenberg")

## Prepare training set

In [4]:
seg = 512
stride = 256

n_sets = 3
n_blocks = 1000
random.seed(42)

tokenizer = AutoTokenizer.from_pretrained("openllama_f16")

## Kafka dataset

In [5]:
with open("chateau.txt", "r", encoding="utf-8") as f:
    kafka = f.read()

# remove page numbers
kafka = re.sub(r"–\s*\d+\s*–\n", "", kafka)

# fix split words
kafka = re.sub(r"-\n", "", kafka)

# remove line breaks
kafka = re.sub(r'\n(?!–)', ' ', kafka)

tok = tokenizer.encode(kafka, add_special_tokens=False)
blocks = [
    tok[i:i+seg]
    for i in range(0, len(tok) - seg + 1, stride)
]
last_complete_end = (len(blocks) - 1) * stride + seg

print(len(blocks))

kafka_json = [
    json.dumps({"text": tokenizer.decode(tok_block)}, ensure_ascii=False)
    for tok_block in blocks
]

Token indices sequence length is longer than the specified maximum sequence length for this model (247169 > 2048). Running this sequence through the model will result in indexing errors


964


## Gutenberg

In [6]:
gutenberg = load_from_disk("gutenberg")
ntexts = len(gutenberg)
print(ntexts)

gutenberg_json = []

for i in range(n_sets):
    gutenberg_json.append([])
    j = 0
    while j < n_blocks:
        item = gutenberg[random.randint(0, ntexts - 1)]
        text = item["text"]
        tok = tokenizer.encode(text, add_special_tokens=False)
        if len(tok) < seg:
            continue
        start = random.randint(0, len(tok) - seg)
        block_tokens = tok[start:start+seg]
        decoded = tokenizer.decode(block_tokens)
        gutenberg_json[i].append(
            json.dumps({"text": decoded}, ensure_ascii=False)
        )
        j += 1

for j in range(3):
    print(f"\n=== extrait {j+1} ===\n")
    print(json.loads(gutenberg_json[0][j])["text"])

5493

=== extrait 1 ===

Saint-Valentin. V. Poésies de Charles d'Orléans,
édit. 1803. (Note de la p. 42.)]

[Note 83:

  Le temps a quitté son manteau
  De vent, de froidure et de pluie...
            CHARLES D'ORLÉANS, édit. 1803, p. 257.

Ces jolis chants d'alouette font penser à la vieille petite chanson,
incomparable de légèreté et de prestesse:

  J'étais petite et simplette
  Quant à l'école on me mit
  Et je n'y ai rien appris...
  Qu'un petit mot d'amourette...
  Et toujours je le redis.
  Depuis qu'ay un bel amy.]

Telle fut en général notre primitive et naturelle France, un peu
légère peut-être pour le sérieux d'aujourd'hui. Telle elle fut en
poésie comme elle est en vins, en femmes. Ceux de nos vins que le
monde aime et recherche comme français, ne sont, il est vrai, qu'un
souffle, mais c'est un souffle d'esprit. La beauté française, non
plus, n'est pas facile à bien saisir; ce n'est ni le beau sang
anglais, ni la régularité italienne; quoi donc? le mouvement, la
grâce, le j

## Final dataset

In [7]:
with open(f"train.jsonl", "w", encoding="utf-8") as f:
    for j, gj in enumerate(gutenberg_json):
        combined = kafka_json + gj
        random.shuffle(combined)
        for line in combined:
            f.write(line + "\n")